In [ ]:
import os
import pandas as pd
from itertools import islice
import torch
from torch.utils.data import DataLoader
from utils.text_metrics import evaluate_all_metrics
from utils.temp_utils import *
from utils.lstm_models import DinoLSTMAttnCaptioner, DinoBiLSTMAttnCaptioner
from utils.chexpert_dataset import CheXpertDataset
from utils.padchest_dataset import PadChestGRDataset

# Data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

os.chdir(os.path.dirname(os.getcwd()))

df = pd.read_csv("Datasets/PadChest-GR/master_table.csv")   # must contain column 'ImageID'
root_dir = "Datasets/PadChest-GR/PadChest_GR"
json_file = "Datasets/PadChest-GR/grounded_reports_20240819.json"

df_train = df[df['split'] == 'train']
df_validation = df[df['split'] == 'validation']
df_test = df[df['split'] == 'test']


IMG_SIZE = 1024
MAX_LEN = 64
NUM_BATCH = 8

tf = dino_image_transform(img_size=IMG_SIZE)

ds_train = PadChestGRDataset(
        dataframe=df_train,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

ds_valid = PadChestGRDataset(
        dataframe=df_validation,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

ds_test = PadChestGRDataset(
        dataframe=df_test,
        root_dir=root_dir,
        json_file=json_file,
        max_txt_len=MAX_LEN,
        image_size=IMG_SIZE,
        normalize=True,
        transform=None,
        return_paths=False,
        sentence_key="sentence_en",
    )

tokenizer = build_tokenizer_from_labels()
pad_id = tokenizer.pad_token_id
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
collate_fn = CaptionCollate(tokenizer, pad_id)

train_loader = DataLoader(ds_train, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(ds_valid, batch_size=NUM_BATCH, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(ds_test, batch_size=NUM_BATCH, shuffle=False, collate_fn=collate_fn)

Using device: cuda


# Model

In [11]:
# DINO ViT-S/16 hidden size is 384 
EMBEDDING_D_IMG = 384
N_PREFIX = (IMG_SIZE // 16) ** 2  # number of visual prefix tokens (including CLS)

model = DinoBiLSTMAttnCaptioner(
    vocab_size=tokenizer.vocab_size,
    d_img=EMBEDDING_D_IMG,
    d_h=512,
    pad_id=pad_id,
    dino_model_id="facebook/dinov3-vits16-pretrain-lvd1689m",
    freeze_dino=True,
).to(device)

# Train Parameters

In [12]:
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), lr=3e-4, weight_decay=1e-2
)
loss = sequence_ce_loss
NUM_EPOCHS = 50
BATCHES_PER_EPOCH = 10

# Training

In [13]:
for epoch in range(NUM_EPOCHS):
    slice_train_loader = islice(train_loader, BATCHES_PER_EPOCH)
    slice_valid_loader = islice(valid_loader, BATCHES_PER_EPOCH)
    train_stats = train_one_epoch(model, slice_train_loader, optimizer, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss, grad_clip=1.0)
    val_stats = evaluate(model, slice_valid_loader, device, pad_id, num_batches=BATCHES_PER_EPOCH, loss_fn=loss)
    print(f"Epoch {epoch + 1}: Train Loss={train_stats['loss']:.4f}, PPL={train_stats['ppl']:.2f} | "
            f"Val Loss={val_stats['val_loss']:.4f}, Val PPL={val_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 10/10 [00:12<00:00,  1.26s/it]


Epoch 1: Train Loss=9.4525, PPL=20879.96 | Val Loss=7.6378, Val PPL=2160.62


Evaluating: 100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


Epoch 2: Train Loss=6.6311, PPL=974.38 | Val Loss=6.0120, Val PPL=437.65


Evaluating: 100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


...

Epoch 48: Train Loss=2.8042, PPL=16.53 | Val Loss=2.9347, Val PPL=18.97


Evaluating: 100%|██████████| 10/10 [00:12<00:00,  1.24s/it]


Epoch 49: Train Loss=2.7922, PPL=16.35 | Val Loss=2.9853, Val PPL=19.96


Evaluating: 100%|██████████| 10/10 [00:12<00:00,  1.24s/it]

Epoch 50: Train Loss=2.7762, PPL=16.07 | Val Loss=3.0019, Val PPL=20.55


# Test Parameters

In [14]:
BATCHES_PER_TEST = 1
GREEDY_DECODE = False
TEST_MAX_LEN = 512
TEST_TOP_P = 0.7
TEST_TEMPERATURE = 0.7

# Test

In [15]:
slice_test_loader = islice(test_loader, BATCHES_PER_TEST)
test_stats = evaluate(model, slice_test_loader, device, pad_id, num_batches=BATCHES_PER_TEST)
print(f"Test Loss={test_stats['val_loss']:.4f}, Test PPL={test_stats['val_ppl']:.2f}")

Evaluating: 100%|██████████| 1/1 [00:01<00:00,  1.28s/it]

Test Loss=2.8450, Test PPL=17.20


# Test Report Generation

In [17]:
with torch.no_grad():
    for pixel_values, ids_loader, paths, raw_labels in train_loader:
        pixel_values = pixel_values.to(device)
        gen_ids = model.generate(
            pixel_values=pixel_values,
            bos_id=bos_id, eos_id=eos_id,
            max_new_tokens=TEST_MAX_LEN, top_p=TEST_TOP_P, temperature=TEST_TEMPERATURE, greedy=GREEDY_DECODE
        )
        print("Predictions (first batch):")
        for i in range(gen_ids.size(0)):
            text_gen = tokenizer.decode(gen_ids[i].tolist())
            text_tgt = tokenizer.decode(ids_loader[i].tolist())
            print(f"\nGEN {i+1}:", text_gen)
            print(f"TGT {i+1}:", text_tgt)
            results = evaluate_all_metrics([text_tgt], [text_gen], evaluation_mode="CheXagent")
            for metric, scores in results.items():
                print(f"{metric}: {scores}")
        del pixel_values, ids_loader, paths, raw_labels, gen_ids
        torch.cuda.empty_cache()
        break

Predictions (first batch):

GEN 1: impossible metastatic disease diseaseopenia2.50 shadows clavicleicleJourpreviousoorganizations কdisappointing multiple multiple multiple multiple bilateral bilateral gynecomastiaia -40% SocialrOx infectious infectious infectious etiologyology cannot be individualizedizedizedalaterallylyly predominant predominant predominant predominant predominant an an infectious infectious - dominantincludingomatosisMIP atchMTT already present present associated associated vertebral ringlyly predominant predominant predominant persistsistspercent etermatriaws field field field field field ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) or masses massesMax prosthesesScotconstraintsoriginating.
TGT 1: diaphragmatic eventration. aortic elongation. study within normality.
Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.43390440940856934]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_E

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Using device: cuda:0
chexbert_f1_weighted: 0.3333333333333333
chexbert_f1_micro: 0.4
chexbert_f1_macro: 0.07142857142857142
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.14710623025894165]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.0]

GEN 3: holterior thickeningening predominantly predominantly predominantly predominantly predominantly predominantly predominantly predominantly predominantly on on 06 Aside noticeable nextnitroprussideslope bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases bases

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.0677642971277237]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.0]

GEN 4: very low inspirator study carried out out infectious infectious infectious infectious etiology cannot be individualizedizedized are appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciated appreciatedMX Believe irritation Drum character character already present present clear clear lines lines lines lines small small amount amountamplitudes bilateral bilateral apical region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region region 

...

Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.1852710098028183]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.007575757575757576]

GEN 5: very low inspiratorygos lobe as as as as as right right main mainSupernatdemonstratingntilation due due to thyroid thyroid thyroid thyroid oceans prominence prominence prominence prominence improvement improvement improvement improvementescalating Gene, more striking are are recognized accordance and colon towards scapulaulaula pneumonic angles fulfilledBlacks Poundicorn compatible compatible itivesbiomarksuperssis.
TGT 5: dorsal scoliosis. metallic structure projected over the left atrial appendage.
Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.4647451341152191]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l:

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.23381765186786652]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.0]

GEN 7: very low inspiratorygos lobe as as as as tracheostomy tube tube approximately approximately one centimeterimeterimeter lingulaula body body body body body body bodyglucocortic YFP[12–astia /5 a a certain certain component component that contains calcium inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside inside in

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


Using device: cuda:0
chexbert_f1_weighted: 0.16666666666666666
chexbert_f1_micro: 0.2857142857142857
chexbert_f1_macro: 0.07142857142857142
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.29835137724876404]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.0]

GEN 8: abscess with with cystic areas inside inside inside inside inside inside inside inside inside it gives thyroid previous studies studies - dominant,, although they they have progressed progressed compared to consider it gives / 2009 probable probable specific residual character character character character characterracheall hernia.
TGT 8: right diaphragmatic lobulations. aortic elongation. osteopenia with wedging of the 11th dorsal vertebra.
Using device: cuda:0
chexbert_f1_weighted: 0.0
chexbert_f1_micro: 0.0
chexbert_f1_macro: 0.0
chexbert_f1_micro_5: 0.0
chexbert_f1_macro_5: 0.0
bertscore_f1: [0.46688786149024963]
radgraph_f1_RG_E: 0.0
radgraph_f1_RG_ER: 0.0
rouge_l: [0.03571428571428571]
